In [100]:
import pandas as pd
import numpy as np
import re
from itertools import accumulate

In [101]:
emails = pd.read_csv('email_topic_modeling.csv',encoding='latin')

In [103]:
emails = emails[2:3]

In [104]:
emails=emails.head(1)

In [105]:
emails_clean = emails[~emails.Content.isnull()]

In [106]:
emails_clean = emails_clean.Content

In [107]:
emails_clean.drop_duplicates(keep='first', inplace=True)

In [108]:
emails_clean=emails_clean.take(np.random.permutation(len(emails_clean))[:1000])

In [109]:
emails_content = pd.DataFrame(emails_clean)

In [110]:
emails_content['index'] = emails_clean.index

In [111]:
emails_content_clean= emails_content.Content.str.replace("[^a-zA-Z ]", "")

In [112]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\amrenkumar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [113]:
def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

doc_sample = emails_content[emails_content['index'] == 0].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

In [114]:
processed_docs = emails_content['Content'].map(preprocess)
processed_docs[:10]

2    [dear, amrendra, kumar, inform, scan, imag, up...
Name: Content, dtype: object

In [115]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 activ
1 amrendra
2 approv
3 claim
4 confirm
5 dear
6 document
7 excel
8 global
9 imag
10 inform


In [116]:
#dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [117]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
#bow_corpus[5]

In [118]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[]


In [122]:
corpus_tfidf.corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 2),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 2),
  (17, 1),
  (18, 1),
  (19, 2),
  (20, 1),
  (21, 1)]]

In [123]:
lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=3, id2word=dictionary, passes=50, workers=2)

C:\Anaconda\lib\site-packages\gensim\models\ldamodel.py:821: RuntimeWarning: invalid value encountered in double_scalars
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


In [124]:
for idx, topic in lda_model.print_topics(num_words=10):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.045*"portal" + 0.045*"inform" + 0.045*"valid" + 0.045*"upload" + 0.045*"success" + 0.045*"subject" + 0.045*"scan" + 0.045*"reimburs" + 0.045*"regard" + 0.045*"proof"
Topic: 1 
Words: 0.045*"portal" + 0.045*"inform" + 0.045*"valid" + 0.045*"upload" + 0.045*"success" + 0.045*"subject" + 0.045*"scan" + 0.045*"reimburs" + 0.045*"regard" + 0.045*"proof"
Topic: 2 
Words: 0.045*"portal" + 0.045*"inform" + 0.045*"valid" + 0.045*"upload" + 0.045*"success" + 0.045*"subject" + 0.045*"scan" + 0.045*"reimburs" + 0.045*"regard" + 0.045*"proof"


In [102]:
for i in emails.Content[2:3]:
    print(i)

Dear Amrendra Kumar,

    Please be informed that your scan image upload towards Reimbursement Claims for Voucher Id - W8428, is done successfully on our portal.
    This is only a confirmation to your activity of scan image upload being done.
    Approval of proofs is subject to validation of documents.
     

 Regards,
 Excelity Global 
